In [1]:
import os
import subprocess
import argparse
import re

### Extract eng subtitle using ffprobe

In [2]:
# test block
random = 'ffprobe -v error -select_streams s -show_entries stream=index:stream_tags=language -of default=noprint_wrappers=1:nokey=1 video_file'
print(f'{random.split()}')

['ffprobe', '-v', 'error', '-select_streams', 's', '-show_entries', 'stream=index:stream_tags=language', '-of', 'default=noprint_wrappers=1:nokey=1', 'video_file']


In [7]:
video_file = 'D:\Seeding\Movies\Prisoners 2013 1080p BluRay DTS x264-DON.mkv'

if not os.path.exists(video_file):
    print("Error: Video file not found")

command = [
    'ffprobe',
    '-v', 'error',
    '-select_streams', 's',
    '-show_entries', 'stream=index:stream_tags=language',
    '-of', 'default=noprint_wrappers=1:nokey=1',
    video_file
]

try:
    # result = subprocess.run(['ffprobe','-show_streams',video_file], capture_output=True, text=True, check=True)
    result = subprocess.run(command, capture_output=True, text=True, check=True)
except subprocess.CalledProcessError as e:
    print("Error: ", e)

# print(result.stderr.split('\n'))
print(f'{result.stderr}')

### Parse FFmpeg output to find English subtitle track

In [8]:
subtitle_tracks = []
pattern = r'[0-9]+'

for line in result.stderr.split('\n'):
    if 'Subtitle' in line:
        
        parts = line.split(',')
        for part in parts:
            print(part)
            if 'rum' in part.lower():
                
                match = re.findall(pattern, part.split(':')[1])
                print(match)
                subtitle_tracks.append(int(match[0]))

In [5]:
if not subtitle_tracks:
    print("No English subtitle track found")
else:
    print(subtitle_tracks)

No English subtitle track found


### Extract subtitles from the first English subtitle track found

In [6]:
subtitle_track = subtitle_tracks[0]
filename, _ = os.path.splitext(video_file)
subtitle_file = f"{filename}.srt"

IndexError: list index out of range

### Mock of the full ffmpeg command before actually runnning it

In [ ]:
print(f'ffmpeg -i {video_file} -map 0:s:{subtitle_track} {subtitle_file}')

### Execute FFmpeg command to extract English subtitles

In [ ]:
try:
    subprocess.run(['ffmpeg', '-i', video_file, '-map', f'0:s:{subtitle_track}', subtitle_file], check=True)
    print("English subtitles extracted successfully")
except subprocess.CalledProcessError as e:
    print(f'Error on extracting subtitle: ',e)